In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

#url = 'https://raw.githubusercontent.com/TheLazyCactus/ML_Project/refs/heads/main/Extra_data.csv'
#df = pd.read_csv(url, sep=";", low_memory =False)

url = 'https://raw.githubusercontent.com/TheLazyCactus/ML_Project/refs/heads/main/ML_Project_safety.csv'
df = pd.read_csv(url, sep=";", low_memory =False)

Need to change the order to get the oldest value first

In [30]:
df

,Year,Company code,FAR total,TRIR total,TRIR company only,TRIR contractor only,LTIR total,LTIR company only,LTIR contractor only
486,2014,ZZ,0,"3,61",0,"5,43",0,0,0
456,2014,W,"2,6",NaN,NaN,NaN,"0,37","0,37","0,37"
455,2014,V,0,"1,41","0,54","1,65","0,39","0,11","0,47"
454,2014,U,"7,03","1,97","1,21","2,2","0,49","0,61","0,46"
453,2014,T,0,"4,47","2,57","6,24","0,5","0,51","0,48"
...,...,...,...,...,...,...,...,...,...
31,2023,GG,"1,06","0,38","0,8","0,32","0,18","0,51","0,13"
30,2023,FF,"1,34","0,98","0,75","1,07","0,18","0,15","0,19"
29,2023,EE,0,"0,46","0,33","0,51","0,18","0,33","0,13"
27,2023,CC,0,"0,2",0,"0,24","0,2",0,"0,24"


In [2]:
df = df.sort_values(by="Year", ascending=True)
#Drop FAR
df.drop("FAR total", axis = 1, inplace=True)

cols = ["TRIR total", "TRIR company only", "TRIR contractor only", "LTIR total", "LTIR company only", "LTIR contractor only" ]  # List of columns to convert
df[cols] = df[cols].replace(',', '.', regex=True).astype(float)

df[cols] = df[cols].astype(float)

df = df.fillna(0)

original_df = df

**EDA**

In [55]:
print(df.shape)
print(df.columns)
print(df.dtypes)
print(df.nunique())
print(df.isna().sum())

(487, 8)
Index(['Year', 'Company code', 'TRIR total', 'TRIR company only',
       'TRIR contractor only', 'LTIR total', 'LTIR company only',
       'LTIR contractor only'],
      dtype='object')
Year                      int64
Company code             object
TRIR total              float64
TRIR company only       float64
TRIR contractor only    float64
LTIR total              float64
LTIR company only       float64
LTIR contractor only    float64
dtype: object
Year                     10
Company code             59
TRIR total              247
TRIR company only       188
TRIR contractor only    279
LTIR total              121
LTIR company only       105
LTIR contractor only    138
dtype: int64
Year                    0
Company code            0
TRIR total              0
TRIR company only       0
TRIR contractor only    0
LTIR total              0
LTIR company only       0
LTIR contractor only    0
dtype: int64


In [3]:
df_total = df[["Year", "Company code", "TRIR total", "LTIR total"]]
df_company = df[["Year", "Company code", "TRIR company only", "LTIR company only"]]
df_contractor = df[["Year", "Company code", "TRIR contractor only", "LTIR contractor only"]]

**Time Serie for TRIR**

*ARIMA*

In [18]:
pip install pmdarima --quiet

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


   ---------------------------------------- 0.0/625.1 kB ? eta -:--:--
    --------------------------------------- 10.2/625.1 kB ? eta -:--:--
   -- ------------------------------------ 41.0/625.1 kB 653.6 kB/s eta 0:00:01
   ------------- -------------------------- 204.8/625.1 kB 1.8 MB/s eta 0:00:01
   -------------------------------- ------- 501.8/625.1 kB 3.2 MB/s eta 0:00:01
   ---------------------------------------- 625.1/625.1 kB 3.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.4/2.8 MB 12.6 MB/s eta 0:00:01
   ------------ --------------------------- 0.8/2.8 MB 10.6 MB/s eta 0:00:01
   ------------------- -------------------- 1.3/2.8 MB 10.6 MB/s eta 0:00:01
   ------------------------- -------------- 1.8/2.8 MB 10.5 MB/s eta 0:00:01
   ------------------------------- -------- 2.2/2.8 MB 10.8 MB/s eta 0:00:01
   -------------------------------------- - 2.7/2.8 MB 10.7 MB/s eta 0:00:01
   ---

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
company_counts = df_total.groupby("Company code")["Year"].count()
# Get companies with 5 or more rows
valid_companies = company_counts[company_counts >= 5].index

# Filter the original DataFrame to keep only those companies
df_total_filtered = df_total[df_total["Company code"].isin(valid_companies)]
df_total_filtered = df_total_filtered.set_index("Year")

In [12]:
df_total_filtered

,Year,Company code,TRIR total,LTIR total
456,2014,W,0.00,0.37
455,2014,V,1.41,0.39
454,2014,U,1.97,0.49
453,2014,T,4.47,0.50
452,2014,S,1.29,0.53
...,...,...,...,...
31,2023,GG,0.38,0.18
30,2023,FF,0.98,0.18
29,2023,EE,0.46,0.18
27,2023,CC,0.20,0.20


In [ ]:
df_total_filtered.nunique()

Year             10
Company code     50
TRIR total      243
LTIR total      121
dtype: int64

In [74]:
df_total_filtered.dtypes

Year              int64
Company code     object
TRIR total      float64
LTIR total      float64
dtype: object

In [5]:
from joblib import Parallel, delayed
import pmdarima as pm
import pandas as pd

# Function to fit ARIMA and return model summary (for Company only)
def fit_arima(group):
    company = group["Company code"].iloc[0]  # Extract the company name
    ts = group["TRIR total"]  # Convert to time series using the 'TRIR total' column


    try:
        # Fit ARIMA model
        model = pm.auto_arima(ts, start_p=1, start_q=1, max_p=3, max_q=3, d=None, 
                              test="adf", seasonal=False, trace=False, error_action="ignore", 
                              suppress_warnings=True, stepwise=True)
        
        print(f"Successfully fitted ARIMA for {company}")  # Add a print statement to verify success
        # Generate predictions and residuals
        preds = model.predict_in_sample()  # In-sample predictions
        residuals = ts.values - preds  # Residuals (actual - predicted)
        return company, model, preds, residuals

    except Exception as e:
        print(f"Error fitting ARIMA for {company}: {str(e)}")
        return company, None, None, None  


# Debugging the fitting process and checking the data
#print(df_total_filtered.groupby("Company code")["Year"].count())  # Number of rows per company

from joblib import Parallel, delayed

models = Parallel(n_jobs=-1)(delayed(fit_arima)(group) for _, group in df_total_filtered.groupby("Company code"))

final_df = pd.DataFrame()

for i, company_code in enumerate(df_total_filtered["Company code"].unique()):
    company_df = df_total_filtered[df_total_filtered["Company code"] == company_code]

    # Check if the model produced predictions
    if models[i][2] is None or models[i][3] is None:
        print(f"⚠️ Warning: No predictions for Company {company_code}. Skipping...")
        continue  # Skip this company

    preds = models[i][2].to_frame(name="Predictions")  
    residuals = models[i][3].to_frame(name="Residuals")  

    # Ensure "Year" is the index
    if "Year" in company_df.columns:
        company_df = company_df.set_index("Year")

    # Debugging: Print shapes before fixing mismatch
    print(f"Company: {company_code}")
    print(f"company_df.shape: {company_df.shape}, preds.shape: {preds.shape}, residuals.shape: {residuals.shape}")

    # Fix length mismatch
    min_len = min(len(company_df), len(preds), len(residuals))
    company_df = company_df.iloc[-min_len:]  
    preds = preds.iloc[-min_len:]  
    residuals = residuals.iloc[-min_len:]  

    # Assign correct index
    preds.index = company_df.index
    residuals.index = company_df.index

    # Concatenate and append
    final_df_rows = pd.concat([company_df, preds, residuals], axis=1)
    final_df = pd.concat([final_df, final_df_rows], axis=0)

print(final_df)



Company: W         
company_df.shape: (10, 3), preds.shape: (10, 1), residuals.shape: (10, 1)
Company: V         
company_df.shape: (10, 3), preds.shape: (10, 1), residuals.shape: (10, 1)
Company: U         
company_df.shape: (9, 3), preds.shape: (10, 1), residuals.shape: (10, 1)
Company: T         
company_df.shape: (9, 3), preds.shape: (10, 1), residuals.shape: (10, 1)
Company: S         
company_df.shape: (9, 3), preds.shape: (10, 1), residuals.shape: (10, 1)
Company: R         
company_df.shape: (10, 3), preds.shape: (10, 1), residuals.shape: (10, 1)
Company: Q         
company_df.shape: (10, 3), preds.shape: (10, 1), residuals.shape: (10, 1)
Company: P         
company_df.shape: (10, 3), preds.shape: (10, 1), residuals.shape: (10, 1)
Company: O         
company_df.shape: (10, 3), preds.shape: (10, 1), residuals.shape: (10, 1)
Company: N         
company_df.shape: (10, 3), preds.shape: (10, 1), residuals.shape: (10, 1)
Company: M         
company_df.shape: (10, 3), preds.shape: (10

In [ ]:
final_df['Predictions'] = final_df['Predictions'].apply(lambda x: max(x, 0)) #remove neg value

In [6]:
final_df

,Company code,TRIR total,LTIR total,Predictions,Residuals
Year,,,,,
2014,W,0.00,0.37,0.000000,4.340000
2015,W,1.41,0.38,6.510021,-2.450021
2016,W,1.17,0.28,3.780000,-1.330000
2017,W,0.76,0.25,0.840000,4.820000
2018,W,3.05,0.28,8.870000,-6.220000
...,...,...,...,...,...
2019,FF,0.84,0.18,1.017046,-0.117046
2020,FF,0.76,0.12,1.384425,0.585575
2021,FF,0.49,0.14,1.902792,-1.292792


In [19]:
## Fit ARIMA Models for each company group
#models = Parallel(n_jobs=-1)(delayed(fit_arima)(group) for _, group in df_total_filtered.groupby("Company code"))
#
#
#final_df = pd.DataFrame()
#
#
#for i, company_code in enumerate(df_total_filtered["Company code"].unique()):
#    company_df = df_total_filtered[df_total_filtered["Company code"] == company_code]
#    preds = models[i][2]
#    residuals = models[i][3]
#    final_df_rows = pd.concat([company_df, preds, residuals], axis=1, ignore_index=False)
#    print(final_df_rows)
#    preds = preds.to_frame(name="Predictions")  # Assign a column name
#    residuals = residuals.to_frame(name="Residuals") 
#    company_df = company_df.set_index("Year")
#    preds = preds.set_index("Year")
#    residuals = residuals.set_index("Year")
#    final_df = pd.concat([final_df, final_df_rows], axis=0)



In [16]:
## Create lag features for LTIR total
#final_df["LTIR_Lag_1"] = final_df.groupby("Company code")["LTIR total"].shift(1)
#final_df["LTIR_Lag_2"] = final_df.groupby("Company code")["LTIR total"].shift(2)
#
## Drop rows with NaN values due to lagging
#final_df = final_df.dropna(subset=["LTIR_Lag_1", "LTIR_Lag_2"])
#
## Optionally, you can fill NaN values instead of dropping them, for example:
#final_df["LTIR_Lag_1"].fillna(0, inplace=True)
#final_df["LTIR_Lag_2"].fillna(final_df["LTIR total"].mean(), inplace=True)

***LTIR Prediction***

**With lag**

In [63]:
# Features for model stacking: TRIR ARIMA predictions and residuals
final_df_reset = final_df.reset_index()
X = final_df_reset[["Year", "TRIR total", "Predictions", "Residuals", "Company code", "LTIR_Lag_1", "LTIR_Lag_2"]] 
X = pd.get_dummies(X, columns=["Company code"], drop_first=True)
y = final_df_reset["LTIR total"]  # Target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
forest = RandomForestRegressor(n_estimators=200,
                             max_depth=10, random_state=42)
forest.fit(X_train, y_train)
pred = forest.predict(X_test)

print("forest score")
print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score (Test)", forest.score(X_test, y_test))
print("R2 score (Train)", forest.score(X_train, y_train))

bagging_reg = BaggingRegressor(DecisionTreeRegressor(max_depth=10),
                               n_estimators=100,
                               max_samples=100,
                                random_state=42)#take 80% of the total samples

bagging_reg.fit(X_train, y_train)
pred = bagging_reg.predict(X_test)

print("Bag score")
print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", bagging_reg.score(X_test, y_test))

gb_reg = GradientBoostingRegressor(max_depth=10,
                                   n_estimators=100,
                                   random_state=42)
gb_reg.fit(X_train, y_train)
pred = gb_reg.predict(X_test)

print("GradBoost score")
print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", gb_reg.score(X_test, y_test))
print("R2 score", gb_reg.score(X_train, y_train))

ada_reg = AdaBoostRegressor(DecisionTreeRegressor(max_depth=10),
                            n_estimators=100,
                            random_state=42)
ada_reg.fit(X_train, y_train)
pred = ada_reg.predict(X_test)
print("AdaBoost score")
print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", mean_squared_error(pred, y_test, squared=False))
print("R2 score", ada_reg.score(X_test, y_test))
print("R2 score", ada_reg.score(X_train, y_train))

MAE 0.062186721718179745
RMSE 0.1130349630646833
R2 score (Test) 0.8952912343428158
R2 score (Train) 0.9428636926020838


d:\Data_analyst\Ironhack\Anaconda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


MAE 0.05734840298023972
RMSE 0.09367253371031917
R2 score 0.9280912099289509


d:\Data_analyst\Ironhack\Anaconda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


MAE 0.08895206808890506
RMSE 0.17667366220266706
R2 score 0.7441996882304353
R2 score 0.9999999992935337


d:\Data_analyst\Ironhack\Anaconda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


MAE 0.06714285714285713
RMSE 0.11749945722844674
R2 score 0.8868566150745893
R2 score 0.9981445344439233


d:\Data_analyst\Ironhack\Anaconda\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


**Cross validation**

In [53]:
#With lag
X = final_df.drop(columns=['LTIR total'])  # Drop target variable
X = pd.get_dummies(X, columns=["Company code"], drop_first=True)
y = final_df['LTIR total']  # Target variable

# Instantiate the Random Forest Regressor model
rf = RandomForestRegressor(n_estimators=200, random_state=42, max_depth= 10)

# Perform cross-validation
cv_results = cross_val_score(rf, X, y, cv=5, scoring='neg_mean_absolute_error')

# cv_results returns the negative of the score (since lower MAE is better)
# Convert it back to positive values
cv_results = -cv_results

# Print the results
print(f"Cross-validation results for MAE: {cv_results}")
print(f"Average MAE: {np.mean(cv_results)}")
print(f"Standard Deviation of MAE: {np.std(cv_results)}")

Cross-validation results for MAE: [0.0710537  0.13101948 0.4430938  0.04985023 0.05046016]
Average MAE: 0.14909547258208491
Standard Deviation of MAE: 0.14995387365117138


**Overfitting?**

Compare MAE cross validation to train MAE
If cross validation MAE too high compare to train MAE, model good for training sata but not for new data -> Overfitting

**Hyper param tuning**

In [ ]:
grid = {"n_estimators": [50, 100, 200,500],
        "estimator__max_leaf_nodes": [250, 500, 1000, None],
        "estimator__max_depth":[10,30,50]}

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
random_reg = RandomForestRegressor(DecisionTreeRegressor())
model = GridSearchCV(estimator = random_reg, param_grid = grid, cv=5)
model.fit(X_train, y_train)

In [ ]:
model.best_params_

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error
pred = random_reg.predict(X_test)

print("MAE", mean_absolute_error(pred, y_test))
print("RMSE", root_mean_squared_error(pred, y_test))
print("R2 score", random_reg.score(X_test, y_test))